## **3. Pondération statistique** (TF-IDF / OKapiBM25)  

https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python  
http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

https://pypi.org/project/rank-bm25/

In [97]:
path = '../04-filtrage/output/'
acteur = 'msss'
tag = 'dependances'

if tag:
    csv_file = acteur + '_' + tag + '_significant-collocations.csv'

else:
    csv_file = acteur + '_significant-collocations.csv'

### **Lire le vocabulaire** (termes retenus au prétraitement)

In [98]:
from pandas import *

with open(path+csv_file, encoding='utf-8') as f:
    csv = read_csv(f).drop(columns = ['Unnamed: 0'])

csv

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF)
0,québec,NOM,204,-,-,138
1,services,NOM,170,-,-,64
2,programme,NOM,143,-,-,106
3,santé,NOM,142,-,-,34
4,répertoire des ressources en dépendances,NOM PRP:det NOM PRP NOM,140,1239.1304527524449,1.911053182078075e-271,138
...,...,...,...,...,...,...
1239,projets financés par le puds,NOM VER:pper PRP DET:ART NOM,3,48.242099720226136,3.767129038526422e-12,3
1240,connaissances entourant,NOM VER:ppre,3,50.28088816177004,1.3324088146915905e-12,3
1241,cohérence avec les orientations,NOM PRP DET:ART NOM,3,52.17742972527093,5.070559804589293e-13,3
1242,cohérence avec les orientations du québec,NOM PRP DET:ART NOM PRP:det NOM,3,30.46543247439436,3.398688332247129e-08,3


In [99]:
vocabulaire = [t.lower() for t in list(csv['Terme'])]

In [100]:
print('On a un vocabulaire de {} formes.'.format(len(vocabulaire)))
vocabulaire

On a un vocabulaire de 1244 formes.


['québec',
 'services',
 'programme',
 'santé',
 'répertoire des ressources en dépendances',
 'ans',
 'répertoires ressources en dépendances',
 'msss répertoires',
 'centre',
 'fiche',
 'fiche de la ressource',
 'fiche de la ressource répertoire',
 'ressource répertoire',
 'ressource répertoire des ressources en dépendances',
 'jours',
 'clientèle',
 'soutien',
 'thérapie',
 'jeunes',
 'aide',
 'approche',
 'durée',
 'capacité',
 'réadaptation',
 'télécopieur',
 'hébergement',
 'services sociaux',
 'durée du programme',
 'évaluation',
 'ressource certifiée en dépendances',
 'ressource certifiée',
 'hommes',
 'réinsertion sociale',
 'réinsertion',
 'aide et soutien',
 'ressource à but non lucratif',
 'groupe',
 'alcool',
 'jours approche',
 'femmes',
 'désintoxication',
 'service',
 'soutien à la désintoxication',
 'hommes et femmes',
 'externe',
 'jeu',
 'centre de réadaptation',
 'prévention',
 'indicateurs',
 'maison',
 'concours',
 'info',
 'toxicomanie',
 'montérégie',
 'problèmes'

### **Lire le corpus**

In [101]:
import os, shutil, re
from pathlib import Path
from os import path
from pandas import *

base_path = '../03-corpus/2-data/1-fr/'
if tag:
    base_path = path.join(base_path, acteur, acteur + '_' + tag + '.csv')

else:
    base_path = path.join(base_path, acteur +  '.csv')
        
with open(base_path, "r", encoding = "UTF-8") as f:
    data = read_csv(base_path, sep=',')
    text = data['text'].tolist()

In [102]:
text = text[:round(len(text))]

nb_docs = len(text)

print("On a donc un corpus de {} documents.".format(nb_docs))

On a donc un corpus de 165 documents.


### **Nettoyage**

In [103]:
corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in text]
    
punct = '[!#$%&•►*+,;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in corpus]
corpus = [re.sub(spaces, ' ', t) for t in corpus]
corpus = [re.sub(punct, ' ', t).replace("' ", "'" ).replace("'", "'") for t in corpus]
corpus = [re.sub(phones, ' STOP ', t) for t in corpus]
corpus = [re.sub(postals, ' STOP ', t) for t in corpus]
corpus = [t.replace("  ", " " ) for t in corpus]

In [104]:
#corpus

In [105]:
sw = []
file_mwesw = '../04-filtrage/mwe_stopwords.txt'
with open (file_mwesw, 'r', encoding='utf-8') as f:
    sw += [t.lower().strip('\n') for t in f.readlines()]

# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    sw += [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    sw += [t.lower().strip('\n') for t in f.readlines()]

sw.append('STOP')

In [106]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')
def tokenize(corpus):
    corpus_tokenized = []
    for doc in corpus:
        output = []
        for t in tagger.tag_text(doc):
            try: 
                output.append((t.split('\t')[0], t.split('\t')[1]))
            except Exception as e:
                output.append(('STOP', 'NAM'))

        tokens = [t[0] for t in output]
        corpus_tokenized.append(tokens)
    return corpus_tokenized

tokenized_corpus = tokenize(corpus)

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import RegexpTokenizer

#toknizer = RegexpTokenizer()

# max_df : ignore words that appear in 85% of documents, 
# min df:  ignore words that appear in less than 1% of documents 
# vocabulary = vocabulaire

# Sans utiliser le vocabulaire
# tfidf = TfidfVectorizer(min_df=0.1, stop_words=None, ngram_range=(2,4), max_df=0.85, use_idf=True)

def identity_tokenizer(text):
    return text


# vocabulary = vocabulaire
tfidf_vectorizer = TfidfVectorizer(tokenizer = identity_tokenizer, vocabulary=vocabulaire, ngram_range=(1,10), stop_words = sw, lowercase=False)
tfidf = tfidf_vectorizer.fit_transform(tokenized_corpus)

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [' ', "'", '-', '.', '0', '1', '2', '3', '4', '5', '7', '8', '9', 'O', 'P', 'S', 'T', '\\', '_', '|', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ñ', 'ó', 'ö', 'ù', 'û', 'œ', '’', '”'] not in stop_words.
  warnings.warn(


In [108]:
features_names = tfidf_vectorizer.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(tfidf.T.todense(), index=features_names, columns=corpus_index).transpose()

In [109]:
df

,québec,services,programme,santé,répertoire des ressources en dépendances,ans,répertoires ressources en dépendances,msss répertoires,centre,fiche,...,besoins identifiés,connaissances entourant les soins de santé,admissibilité les projets,tcc,mesures adoptées,projets financés par le puds,connaissances entourant,cohérence avec les orientations,cohérence avec les orientations du québec,charlemagne
0,0.000000,0.000000,0.000000,0.030765,0.0,0.175809,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.068248,0.057170,0.109958,0.257739,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.068058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.154586,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.000000,0.000000,0.000000,0.030765,0.0,0.175809,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.0,0.136883,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,0.019123,0.288339,0.000000,0.247602,0.0,0.000000,0.0,0.0,0.028540,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,0.019073,0.287581,0.000000,0.246950,0.0,0.000000,0.0,0.0,0.028465,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162,0.019123,0.288339,0.000000,0.247602,0.0,0.000000,0.0,0.0,0.028540,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,0.019073,0.287581,0.000000,0.246950,0.0,0.000000,0.0,0.0,0.028465,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
terms_tfidf = {term: df[term].max() for term in df}
zeros = {term : value for term, value in terms_tfidf.items() if terms_tfidf[term] == 0}

In [111]:
len(zeros) / len(vocabulaire) *100

51.527331189710615

In [112]:
terms_tfidf

{'québec': 0.4991429138627616,
 'services': 0.38232192708855034,
 'programme': 0.21582474239867713,
 'santé': 0.6052399106307177,
 'répertoire des ressources en dépendances': 0.0,
 'ans': 0.22604265897151893,
 'répertoires ressources en dépendances': 0.0,
 'msss répertoires': 0.15743243210257182,
 'centre': 0.43439448352683596,
 'fiche': 0.17383468129944152,
 'fiche de la ressource': 0.0,
 'fiche de la ressource répertoire': 0.0,
 'ressource répertoire': 0.1760359560497918,
 'ressource répertoire des ressources en dépendances': 0.0,
 'jours': 0.43231931694540315,
 'clientèle': 0.24579963970139315,
 'soutien': 0.3941175298470955,
 'thérapie': 0.4386212576344256,
 'jeunes': 0.3925665460217871,
 'aide': 0.4602160779949883,
 'approche': 0.3485990742178174,
 'durée': 0.20403648628509477,
 'capacité': 0.19988662209831093,
 'réadaptation': 0.41043660756230677,
 'télécopieur': 0.2026380243382865,
 'hébergement': 0.3681485037961241,
 'services sociaux': 0.4051306630200102,
 'durée du programme'

In [113]:
terms_weighted = DataFrame(terms_tfidf.items(), columns=['Terme', 'TF-IDF'])

In [114]:
terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

terms_weighted = terms_weighted.drop_duplicates(keep='first')
terms_weighted

,Terme,TF-IDF
948,lien,0.953224
171,bas-saint-laurent,0.762762
203,nord-du-québec,0.760037
308,côte-nord,0.749212
87,lanaudière,0.713498
...,...,...
636,accent sur l'engagement,0.000000
637,accent sur l'engagement social,0.000000
639,ans aux risques,0.000000
640,campagne campagne sur l'alcool,0.000000


In [115]:
terms_weighted = pd.merge(csv, terms_weighted, on='Terme').drop_duplicates(
  subset = ['Terme', 'Fréquence (TF)'],
  keep = 'first').reset_index(drop = True)

## **OKapi BM25**
https://hal.archives-ouvertes.fr/hal-00760158 

In [116]:
from rank_bm25 import BM25Okapi

In [117]:
bm25 = BM25Okapi(tokenized_corpus)

In [118]:
vocabulaire

['québec',
 'services',
 'programme',
 'santé',
 'répertoire des ressources en dépendances',
 'ans',
 'répertoires ressources en dépendances',
 'msss répertoires',
 'centre',
 'fiche',
 'fiche de la ressource',
 'fiche de la ressource répertoire',
 'ressource répertoire',
 'ressource répertoire des ressources en dépendances',
 'jours',
 'clientèle',
 'soutien',
 'thérapie',
 'jeunes',
 'aide',
 'approche',
 'durée',
 'capacité',
 'réadaptation',
 'télécopieur',
 'hébergement',
 'services sociaux',
 'durée du programme',
 'évaluation',
 'ressource certifiée en dépendances',
 'ressource certifiée',
 'hommes',
 'réinsertion sociale',
 'réinsertion',
 'aide et soutien',
 'ressource à but non lucratif',
 'groupe',
 'alcool',
 'jours approche',
 'femmes',
 'désintoxication',
 'service',
 'soutien à la désintoxication',
 'hommes et femmes',
 'externe',
 'jeu',
 'centre de réadaptation',
 'prévention',
 'indicateurs',
 'maison',
 'concours',
 'info',
 'toxicomanie',
 'montérégie',
 'problèmes'

In [119]:
#tokenizer = RegexpTokenizer(r"\w\'|\w+")
## Revoir ça
tokenized_queries = [t.split() for t in set(vocabulaire)]

features_names = [t for t in set(vocabulaire)]
corpus_index = [corpus.index(n) for n in corpus]

tab = [bm25.get_scores(query) for query in tokenized_queries]
df = pd.DataFrame(tab, index=features_names, columns=corpus_index).transpose()

In [120]:
print(len(tokenized_queries))

tokenized_queries

1244


[['cohérence'],
 ['évaluation', 'des', 'avantages', "d'ensemble"],
 ['milieu', 'des', 'soins', 'de', 'santé'],
 ['admissibilité', 'concours'],
 ['cdf', 'pour', 'participer', 'au', 'concours'],
 ['responsabilité', 'de', 'santé'],
 ['mesures', 'adoptées'],
 ['délivrance', "d'une", 'attestation'],
 ['innovation', 'réel'],
 ['critère', 'de', 'consultation'],
 ['jeux'],
 ['éducation'],
 ['entourage', 'clientèle'],
 ['capacité', 'des', 'jeunes'],
 ['besoins'],
 ['secrétariat'],
 ['centre', 'dollard-cormier'],
 ['répertoire', 'des', 'indicateurs', 'de', 'gestion', 'en', 'santé'],
 ['indicateurs', 'de', 'gestion', 'le', 'répertoire'],
 ['dépendance', 'en', 'centre', 'de', 'réadaptation'],
 ['modifications', 'en', 'fonction', 'des', 'changements'],
 ['saguenay'],
 ['quiz', 'pour', 'les', 'jeunes'],
 ['tribunaux'],
 ['hébergement', 'en', 'toxicomanie', 'ou', 'en', 'jeu', 'pathologique'],
 ['annulation'],
 ['volet'],
 ['évaluation', 'des', 'conducteurs'],
 ['hasard'],
 ['succès', 'réel', 'ou', 'a

In [121]:
df

,cohérence,évaluation des avantages d'ensemble,milieu des soins de santé,admissibilité concours,cdf pour participer au concours,responsabilité de santé,mesures adoptées,délivrance d'une attestation,innovation réel,critère de consultation,...,matériel de l'édition 2021. guide,cession,usagers en attente d'un service,sensibilisation,problèmes pouvant,engagement,promotion du concours pour la province du québec,ressource certifiée en dépendances réinsertion,objet de modifications,longueil
0,0.000000,1.769479,4.590197,0.000000,4.723972,2.820718,0.0,0.0,0.0,2.025493,...,10.835822,0.0,0.543273,2.929844,0.000000,4.076623,5.270640,2.032622,2.025493,0.0
1,2.555238,1.861665,6.242020,2.083454,3.444793,6.664265,0.0,0.0,0.0,1.916315,...,1.916315,0.0,1.827784,0.000000,3.956894,0.000000,7.906225,2.842783,7.394312,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,3.074957,1.966464,0.000000,0.0
0,0.000000,1.769479,4.590197,0.000000,4.723972,2.820718,0.0,0.0,0.0,2.025493,...,10.835822,0.0,0.543273,2.929844,0.000000,4.076623,5.270640,2.032622,2.025493,0.0
4,0.000000,1.702959,3.768992,0.000000,3.237194,2.066032,0.0,0.0,0.0,2.066032,...,7.394080,0.0,1.414592,3.004263,0.000000,0.000000,5.597615,2.706162,2.066032,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,0.000000,1.848586,6.321145,0.000000,1.998735,4.472558,0.0,0.0,0.0,6.831387,...,2.072425,0.0,9.530900,0.000000,0.000000,0.000000,6.371620,2.613538,2.072425,0.0
159,0.000000,1.848586,6.321145,0.000000,1.998735,4.472558,0.0,0.0,0.0,6.831387,...,2.072425,0.0,9.530900,0.000000,0.000000,0.000000,6.371620,2.613538,2.072425,0.0
162,0.000000,1.848586,6.321145,0.000000,1.998735,4.472558,0.0,0.0,0.0,6.831387,...,2.072425,0.0,9.530900,0.000000,0.000000,0.000000,6.371620,2.613538,2.072425,0.0
163,0.000000,1.848586,6.321145,0.000000,1.998735,4.472558,0.0,0.0,0.0,6.831387,...,2.072425,0.0,9.530900,0.000000,0.000000,0.000000,6.371620,2.613538,2.072425,0.0


In [122]:
#df.to_csv(base_path + titre + '_matrice-OkapiBM25.csv') # Si on veut avoir la matrice (mais le fichier peut être très volumineux)

In [123]:
terms_okapi = {term: df[term].max() for term in df}

In [124]:
terms_okapi

{'cohérence': 4.120591721273721,
 "évaluation des avantages d'ensemble": 4.928657271140694,
 'milieu des soins de santé': 7.865993213304201,
 'admissibilité concours': 6.3912732252722835,
 'cdf pour participer au concours': 16.119700731152072,
 'responsabilité de santé': 6.664265348252382,
 'mesures adoptées': 1.7286947684282707,
 "délivrance d'une attestation": 9.305412508518764,
 'innovation réel': 5.161395336368443,
 'critère de consultation': 6.831386555870967,
 'jeux': 3.686676329677264,
 'éducation': 1.870523632265419,
 'entourage clientèle': 3.684335982956455,
 'capacité des jeunes': 4.323467691344429,
 'besoins': 3.8384305212387844,
 'secrétariat': 2.197176440629542,
 'centre dollard-cormier': 3.0227999157242387,
 'répertoire des indicateurs de gestion en santé': 21.450672768120008,
 'indicateurs de gestion le répertoire': 16.308089427406884,
 'dépendance en centre de réadaptation': 7.977659882302984,
 'modifications en fonction des changements': 11.090975448413285,
 'saguenay'

In [125]:
tab = DataFrame(terms_okapi.items(), columns=['Terme', 'OkapiBM25'])
tab

,Terme,OkapiBM25
0,cohérence,4.120592
1,évaluation des avantages d'ensemble,4.928657
2,milieu des soins de santé,7.865993
3,admissibilité concours,6.391273
4,cdf pour participer au concours,16.119701
...,...,...
1239,engagement,4.076623
1240,promotion du concours pour la province du québec,18.503176
1241,ressource certifiée en dépendances réinsertion,6.131816
1242,objet de modifications,7.394312


In [126]:
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

tab

,Terme,OkapiBM25
298,indicateurs de gestion utilisés par le ministè...,28.082703
316,concours pour le prix du conseil de la fédération,24.217824
245,attestation temporaire de conformité en dépend...,24.082217
859,délivrance d'une attestation temporaire de con...,24.006155
265,changements apportés au puds par santé canada,23.580131
...,...,...
829,thérapie,0.536187
823,centre d'hébergement,0.345756
483,centre,0.345756
277,hommes,0.305342


In [127]:
terms_weighted

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),TF-IDF
0,québec,NOM,204,-,-,138,0.499143
1,services,NOM,170,-,-,64,0.382322
2,programme,NOM,143,-,-,106,0.215825
3,santé,NOM,142,-,-,34,0.605240
4,répertoire des ressources en dépendances,NOM PRP:det NOM PRP NOM,140,1239.1304527524449,1.911053182078075e-271,138,0.000000
...,...,...,...,...,...,...,...
1239,projets financés par le puds,NOM VER:pper PRP DET:ART NOM,3,48.242099720226136,3.767129038526422e-12,3,0.000000
1240,connaissances entourant,NOM VER:ppre,3,50.28088816177004,1.3324088146915905e-12,3,0.024434
1241,cohérence avec les orientations,NOM PRP DET:ART NOM,3,52.17742972527093,5.070559804589293e-13,3,0.000000
1242,cohérence avec les orientations du québec,NOM PRP DET:ART NOM PRP:det NOM,3,30.46543247439436,3.398688332247129e-08,3,0.000000


In [128]:
tab = pd.merge(terms_weighted, tab, on="Terme")
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

In [129]:
base_path = '../05-transformation/'

In [130]:
if tag:
    file_path = base_path + acteur + '_' + tag + '_weighting_OKapiBM25--NO-VOC.csv'

else: 
    file_path = base_path + acteur  + '_weighting_OKapiBM25-NO-VOC.csv'
tab.to_csv(file_path)

In [131]:
tab

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),TF-IDF,OkapiBM25
462,indicateurs de gestion utilisés par le ministè...,NOM PRP NOM VER:pper PRP DET:ART NOM PRP DET:A...,6,65.44632415216947,5.971852030412258e-16,6,0.000000,28.082703
873,concours pour le prix du conseil de la fédération,NOM PRP DET:ART NOM PRP:det NOM PRP DET:ART NOM,3,48.242099720226136,3.767129038526422e-12,3,0.000000,24.217824
703,attestation temporaire de conformité en dépend...,NOM ADJ PRP NOM PRP NOM,4,46.02897026761987,1.1651718413257427e-11,2,0.000000,24.082217
702,délivrance d'une attestation temporaire de con...,NOM PRP DET:ART NOM ADJ PRP NOM,4,59.15381473526139,1.45816396710764e-14,2,0.000000,24.006155
1085,changements apportés au puds par santé canada,NOM VER:pper PRP:det NOM PRP NOM NOM,3,45.517939671890566,1.512471780867477e-11,3,0.000000,23.580131
...,...,...,...,...,...,...,...,...
17,thérapie,NOM,109,-,-,71,0.438621,0.536187
8,centre,NOM,128,-,-,93,0.434394,0.345756
444,centre d'hébergement,NOM PRP NOM,6,62.22222164096381,3.0680628480794446e-15,5,0.000000,0.345756
31,hommes,NOM,75,-,-,74,0.227111,0.305342
